In [40]:
from src.models.image_models.image_attention_model.residual_attn_model import ResidualAttentionNetwork

In [41]:
!ls

README.md                  image_classification.ipynb
Untitled1.ipynb            image_folder_name.txt
attention_model.ipynb      mmf_hm_example.ipynb
attn_model.ipynb           model_plot.png
hateful_img_meme.model     results_montage.png
hateful_nothateful         src


In [42]:
from src.models.image_models.load_config import config_loader

In [43]:
cat src/config/model_config.ini

[DatasetConfig]
base_path = "hateful_nothateful"
train_path = "hateful_nothateful/training"
validation_path = "hateful_nothateful/validation"
test_path = "hateful_nothateful/testing"

[BaseConfig]
classes = ["hateful", "not_hateful"]


[ImageModelConfig]
[Resnet152]
lr = 1e-4
batch_size = 10
num_epochs = 50
trained_model_save_path = "trained_models/resnet152.model"


In [44]:
config_file = "src/config/model_config.ini"

config = config_loader(config_file)

config


{'DatasetConfig': {'base_path': 'hateful_nothateful',
  'train_path': 'hateful_nothateful/training',
  'validation_path': 'hateful_nothateful/validation',
  'test_path': 'hateful_nothateful/testing'},
 'BaseConfig': {'classes': ['hateful', 'not_hateful']},
 'ImageModelConfig': {},
 'Resnet152': {'lr': 0.0001,
  'batch_size': 10,
  'num_epochs': 50,
  'trained_model_save_path': 'trained_models/resnet152.model'}}

In [45]:
base_path = config['DatasetConfig']['base_path']
train_path = config['DatasetConfig']['train_path']
validation_path = config['DatasetConfig']['validation_path']
test_path = config['DatasetConfig']['test_path']

In [46]:
# data_src_path = "data/data"
data_src_path = "data/data"

!echo $data_src_path

data/data


In [47]:
# !python3 src/data_preprocessing/prepare_dataset.py -s $data_src_path -d $base_path

In [48]:
ls hateful_nothateful/training/hateful/ | head -n 10

01235.png
01247.png
01256.png
01258.png
01269.png
01295.png
01349.png
01395.png
01436.png
01467.png


In [49]:
## SETUP PARAMS

lr = config['Resnet152']['lr']
batch_size = config['Resnet152']['batch_size']
num_epochs = config['Resnet152']['num_epochs']
num_epochs = 10
## Parameters
print("####### Model Parameters #######")
print (f"Learning_rate = {lr}")
print (f"batch_size = {batch_size}")
print (f"num_epochs = {num_epochs}")

print("####### -------------- #######")

####### Model Parameters #######
Learning_rate = 0.0001
batch_size = 10
num_epochs = 10
####### -------------- #######


In [50]:
from imutils import paths
paths.list_images(train_path)
totalTrain = len(list(paths.list_images(train_path)))
totalVal = len(list(paths.list_images(validation_path)))
totalTest = len(list(paths.list_images(test_path)))

In [51]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet152
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import tensorflow as tf
from tensorflow.python.keras.engine import data_adapter

print("initializing the training data augmentation object")
# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")
# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
print("initializing the val data augmentation object")
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects

# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean

print("initializing the training generator")
trainGen = trainAug.flow_from_directory(
    train_path,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=batch_size)
print("initializing the val generator")
valGen = valAug.flow_from_directory(
    validation_path,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size)
# initialize the testing generator
print("initializing the testing generator")
testGen = valAug.flow_from_directory(
    test_path,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=1)

initializing the training data augmentation object
initializing the val data augmentation object
initializing the training generator
Found 8500 images belonging to 2 classes.
initializing the val generator
Found 500 images belonging to 2 classes.
initializing the testing generator
Found 540 images belonging to 2 classes.


In [52]:
# baseModel = ResNet152(weights="imagenet", include_top=False,
#                       input_tensor=Input(shape=(224, 224, 3)))

In [53]:
for layer in baseModel.layers[:-1]:
    layer.trainable = False

In [55]:
# headModel = baseModel.output

In [56]:
# for layer in baseModel.layers:
#     print(layer.name ,layer.trainable)

In [57]:
headModel

<tf.Tensor 'conv5_block3_out/Relu_1:0' shape=(None, 7, 7, 2048) dtype=float32>

In [58]:
model = ResidualAttentionNetwork((224,224,3), 1, activation='sigmoid').build_model()

In [59]:
# model.summary()

In [60]:
num_epochs, lr

(10, 0.0001)

In [61]:
opt = Adam(lr=lr, decay=lr / num_epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])
# train the model


In [62]:
print("[INFO] training model...")
H = model.fit(
    x=trainGen,
    steps_per_epoch=totalTrain // batch_size,
    validation_data=valGen,
    validation_steps=totalVal // batch_size,
    epochs=num_epochs)

[INFO] training model...
Epoch 1/10
  2/850 [..............................] - ETA: 2:40:28 - loss: 396.7476 - accuracy: 0.5000

KeyboardInterrupt: 